In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
import scipy.sparse as sps
from datetime import datetime

import numpy as np
from operator import itemgetter
import optuna as op

In [10]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".", "/result_experiments/__Temp_SVDFeature/")))

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

print(f"Train size: {URM_train.getnnz()},  Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

Train size: 1274929,  Val size: 224987, Test size: 264691
EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [6]:
from libs.Recommenders.MatrixFactorization.SVDFeatureRecommender import SVDFeature

In [13]:
def objective_function_SVDFeature(optuna_trial):
    # Extract hyperparameters from the Optuna trial
    num_factors = optuna_trial.suggest_int('num_factors', 10, 1000, log=True)
    learning_rate = optuna_trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    user_reg = optuna_trial.suggest_float('user_reg', 1e-6, 1e-1, log=True)
    item_reg = optuna_trial.suggest_float('item_reg', 1e-6, 1e-1, log=True)
    user_bias_reg = optuna_trial.suggest_float('user_bias_reg', 1e-6, 1e-1, log=True)
    item_bias_reg = optuna_trial.suggest_float('item_bias_reg', 1e-6, 1e-1, log=True)
    epochs = optuna_trial.suggest_int('epochs', 10, 11)
    
    # Initialize the recommender with the training data
    recommender_instance = SVDFeature(URM_train, ICM=ICM_all)
    
    # Fit the model using the sampled hyperparameters
    recommender_instance.fit(
        epochs=epochs,
        num_factors=num_factors,
        learning_rate=learning_rate,
        user_reg=user_reg,
        item_reg=item_reg,
        user_bias_reg=user_bias_reg,
        item_bias_reg=item_bias_reg,
    )
    
    # Evaluate the recommender on validation data
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    # Capture the evaluation metrics
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value for further analysis
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Optimize based on MAP
    return trial_results["MAP"]


In [14]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [16]:
optuna_study_SVDFeature = op.create_study(direction="maximize")
        
save_results_SVDFeature = SaveResults("result_experiments/SVDFeature/results_optuna.csv")
        
optuna_study_SVDFeature.optimize(objective_function_SVDFeature,
                      callbacks=[save_results_SVDFeature],
                      n_trials = 400)

[I 2025-01-06 22:32:02,255] A new study created in memory with name: no-name-bc7f4a5a-afa0-452c-b4c3-50883a5a211e
[W 2025-01-06 22:32:02,256] Trial 0 failed with parameters: {'num_factors': 112, 'learning_rate': 0.03651966971590191, 'user_reg': 5.326243186557064e-05, 'item_reg': 6.910614910994465e-05, 'user_bias_reg': 0.020644265222944394, 'item_bias_reg': 0.0002803817653829934, 'epochs': 11} because of the following error: TypeError('super(type, obj): obj must be an instance or subtype of type').
Traceback (most recent call last):
  File "/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/tr/d1rx7zt15lzgkp2yc1nwbg100000gn/T/ipykernel_69406/3313494516.py", line 12, in objective_function_SVDFeature
    recommender_instance = SVDFeature(URM_train, ICM=ICM_all)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/safuan/

TypeError: super(type, obj): obj must be an instance or subtype of type